In [ ]:
import sys
import os
import pandas as pd
import numpy as np

# プロジェクトルートをパスに追加
sys.path.append('..')

from configs.config import *
from src.util import Logger

## 1. データ読み込み（サンプル数を制限して高速化）

In [ ]:
# 訓練データ（最初の100件のみ）
train_meta = pd.read_csv(RAW_DATA_DIR_NAME / 'input' / 'atmaCup22_metadata' / 'train_meta.csv')
train_meta_sample = train_meta.head(100).copy()

# テストデータ（最初の20件のみ）
test_meta = pd.read_csv(RAW_DATA_DIR_NAME / 'input' / 'atmaCup22_metadata' / 'test_meta.csv')
test_meta_sample = test_meta.head(20).copy()

print(f"Train samples: {len(train_meta_sample)}")
print(f"Test samples: {len(test_meta_sample)}")
print(f"Unique labels: {train_meta_sample['label_id'].nunique()}")

## 2. 旧方式（後方互換性ラッパー経由）

In [ ]:
# 旧importでも動作するか確認
from src.model_resnet import ModelResNet50

logger = Logger('../logs/test_refactored.log')

# Prototype法（method='prototype'がデフォルト）
params_prototype = {
    'model_name': 'resnet50',
    'method': 'prototype',
    'threshold': 0.5,
    'min2_threshold': 0.3,
    'batch_size': 16,
    'num_workers': 0,
    'use_cache': True
}

print("\n=== 旧方式: ModelResNet50 with method='prototype' ===")
model_old = ModelResNet50('test_old_prototype', params_prototype, '../models', logger)
print(f"Type: {type(model_old)}")
print(f"Class name: {model_old.__class__.__name__}")

In [ ]:
# 学習
model_old.train(train_meta_sample)

In [ ]:
# 予測
pred_old = model_old.predict(test_meta_sample)
print(f"\nPrediction shape: {pred_old.shape}")
print(f"Predictions:\n{pred_old.head(10)}")
print(f"\nValue counts:\n{pred_old['label_id'].value_counts()}")

## 3. 新方式（直接import）: Prototype法

In [ ]:
from src.model_resnet_prototype import ModelResNet50Prototype

print("\n=== 新方式: ModelResNet50Prototype ===")
model_proto = ModelResNet50Prototype('test_new_prototype', params_prototype, '../models', logger)
print(f"Type: {type(model_proto)}")
print(f"Class name: {model_proto.__class__.__name__}")

In [ ]:
# 学習
model_proto.train(train_meta_sample)

In [ ]:
# 予測
pred_proto = model_proto.predict(test_meta_sample)
print(f"\nPrediction shape: {pred_proto.shape}")
print(f"Predictions:\n{pred_proto.head(10)}")
print(f"\nValue counts:\n{pred_proto['label_id'].value_counts()}")

In [ ]:
# 旧方式と新方式で結果が一致するか確認
print("\n=== 旧方式 vs 新方式の比較 ===")
print(f"Predictions match: {(pred_old['label_id'] == pred_proto['label_id']).all()}")
if not (pred_old['label_id'] == pred_proto['label_id']).all():
    diff = pred_old[pred_old['label_id'] != pred_proto['label_id']]
    print(f"Differences: {len(diff)} samples")

## 4. 新方式: KNN法

In [ ]:
from src.model_resnet_knn import ModelResNet50KNN

params_knn = {
    'model_name': 'resnet50',
    'k': 3,
    'threshold': 0.5,
    'min2_threshold': 0.3,
    'batch_size': 16,
    'num_workers': 0,
    'use_cache': True
}

print("\n=== 新方式: ModelResNet50KNN ===")
model_knn = ModelResNet50KNN('test_new_knn', params_knn, '../models', logger)
print(f"Type: {type(model_knn)}")
print(f"Class name: {model_knn.__class__.__name__}")

In [ ]:
# 学習
model_knn.train(train_meta_sample)

In [ ]:
# 予測
pred_knn = model_knn.predict(test_meta_sample)
print(f"\nPrediction shape: {pred_knn.shape}")
print(f"Predictions:\n{pred_knn.head(10)}")
print(f"\nValue counts:\n{pred_knn['label_id'].value_counts()}")

## 5. ラッパー経由でKNN法も使えるか確認

In [ ]:
from src.model_resnet import ModelResNet50

params_knn_wrapper = params_knn.copy()
params_knn_wrapper['method'] = 'knn'

print("\n=== 旧方式: ModelResNet50 with method='knn' ===")
model_old_knn = ModelResNet50('test_old_knn', params_knn_wrapper, '../models', logger)
print(f"Type: {type(model_old_knn)}")
print(f"Class name: {model_old_knn.__class__.__name__}")

In [ ]:
# 学習
model_old_knn.train(train_meta_sample)

In [ ]:
# 予測
pred_old_knn = model_old_knn.predict(test_meta_sample)
print(f"\nPrediction shape: {pred_old_knn.shape}")
print(f"Predictions:\n{pred_old_knn.head(10)}")

In [ ]:
# KNN新旧で結果が一致するか
print("\n=== KNN: 旧方式 vs 新方式の比較 ===")
print(f"Predictions match: {(pred_old_knn['label_id'] == pred_knn['label_id']).all()}")

## 6. 閾値チューニング機能のテスト

In [ ]:
# Prototypeモデルで閾値チューニング
print("\n=== 閾値チューニング（Prototype） ===")
pred_custom = model_proto.predict_with_custom_threshold(threshold=0.7, min2_threshold=0.4)
print(f"Custom threshold predictions:")
print(pd.Series(pred_custom).value_counts())

In [ ]:
# KNNモデルで閾値チューニング
print("\n=== 閾値チューニング（KNN） ===")
pred_custom_knn = model_knn.predict_with_custom_threshold(threshold=0.7, k=5)
print(f"Custom threshold predictions (k=5):")
print(pd.Series(pred_custom_knn).value_counts())

## 7. まとめ

### テスト結果
- ✅ 旧方式（ラッパー経由）で後方互換性を保ちつつ動作
- ✅ 新方式（直接import）で各クラスが独立して動作
- ✅ Prototype法とKNN法で結果が異なることを確認
- ✅ 閾値チューニング機能も正常動作

### 移行ガイド
既存のNotebookは変更不要（ラッパーが自動振り分け）。
新規コードでは以下のように直接importを推奨：

```python
# Prototype法
from src.model_resnet_prototype import ModelResNet50Prototype
model = ModelResNet50Prototype(run_fold_name, params, out_dir, logger)

# KNN法
from src.model_resnet_knn import ModelResNet50KNN
model = ModelResNet50KNN(run_fold_name, params, out_dir, logger)
```